In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from pydantic import BaseModel, Field
load_dotenv()
import warnings
warnings.filterwarnings(action="ignore")

In [2]:
os.environ["REQUESTS_CA_BUNDLE"] = r"../../ca-bundle-full.crt"

In [3]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
) 

In [4]:
pay_slip_data = """
EMPLOYEE CODE	1004567
NAME	Rahul Raj
BANK NAME	NEW BANK OF INDIA
BANK ACCOUNT NUMBER	123456789101
PAN	ABCDE1234F
PF ACCOUNT NO.	AP/HYD/98765/4321
PAID DAYS	31
EARNINGS	ACTUALS	AMOUNT	YTD
BASIC	24,765.00	24,765.00	48,705.00
HRA	9,906.00	9,906.00	19,482.00
CONVEYANCE	1,600.00	1,600.00	3,147.00
SPECIAL ALLOWANCE	31,985.00	31,985.00	62,904.00
 	 	 	 
Gross Earnings	68,256.00	68,256.00	134,238.00
DEDUCTIONS	AMOUNT	YTD
PF AMOUNT	2,972.00	5,845.00
PROFESSIONAL TAX	200.00	400.00
INCOME TAX ON SALARY	1,167.00	1,167.00
LABOUR WELFARE FUND	2.00	2.00
FOOD RECOVERY	540.00	870.00
Gross Deductions	4,881.00	8,284.00
Net Transfer	: 63,375.00
IN WORDS	: Rupees Sixty Three Thousand Three Hundred And Seventy Five Only
"""

In [5]:
human_text = "{instruction}"
message = HumanMessagePromptTemplate.from_template(human_text)
prompt = ChatPromptTemplate.from_messages([message])

chain = prompt | llm 
result = chain.invoke({"instruction":"""You are an expert in finance analysis. I will be providing you a pay slip data in three backticks. Get me the below data.\n
              1. Employee Name
              2. Bank Name
              3. Gross Earnings
              4. Income Tax on Salary
              ```"""+pay_slip_data+"```"})

In [6]:
print(result.content)

Based on the provided pay slip data, the requested information is as follows:

1. Employee Name: Rahul Raj
2. Bank Name: NEW BANK OF INDIA
3. Gross Earnings: 68,256.00
4. Income Tax on Salary: 1,167.00


In [7]:
result = chain.invoke({"instruction":"What is the percentage of Income Tax on Salary compared to Gross Earnings"})
print(result.content)

The percentage of income tax on salary compared to gross earnings varies depending on the country and the individual's income level. In general, income tax rates are progressive, meaning they increase as income levels rise. 

For example, in the United States, the income tax rate ranges from 10% to 37% for individuals, depending on their income bracket. However, there are also various deductions, credits, and exemptions that can affect the final amount of income tax owed.

In the United Kingdom, the income tax rates for the tax year 2021/2022 are as follows:
- 0% on earnings up to £12,570
- 20% on earnings between £12,571 and £50,270
- 40% on earnings between £50,271 and £150,000
- 45% on earnings over £150,000

Again, these rates can be subject to various deductions and allowances.

It is important to note that these rates can change over time and can vary significantly between different countries. It is always advisable to consult with a tax professional or refer to the tax authority

In [8]:
conversation = ConversationChain(
    llm=llm,
    verbose=False,
    memory=ConversationBufferMemory()
)

result = conversation.predict(input="""You are an expert in finance analysis. I will be providing you a pay slip data in three backticks. Get me the below data.\n
              1. Employee Name
              2. Bank Name
              3. Gross Earnings
              4. Income Tax on Salary
              ```"""+pay_slip_data+"```")
print(result)

Sure! I have analyzed the pay slip data you provided. Here are the details you requested:

1. Employee Name: Rahul Raj
2. Bank Name: NEW BANK OF INDIA
3. Gross Earnings: Rs. 68,256.00
4. Income Tax on Salary: Rs. 1,167.00

Is there anything else you would like to know about the pay slip data?


In [9]:
result = conversation.predict(input="What is the percentage of Income Tax on Salary compared to Gross Earnings")
print(result)

To calculate the percentage of Income Tax on Salary compared to Gross Earnings, we can use the formula:

Percentage = (Income Tax on Salary / Gross Earnings) * 100

In this case, the Income Tax on Salary is Rs. 1,167.00 and the Gross Earnings is Rs. 68,256.00.

So, the percentage of Income Tax on Salary compared to Gross Earnings is:

(1,167.00 / 68,256.00) * 100 ≈ 1.71%

Please note that the calculated percentage is approximate. Let me know if there's anything else I can assist you with!


In [10]:
result = conversation.predict(input="""For the below components calculate the percentage w.r.t Basic\n 1. HRA\n 2.Conveyance\n3.Special Allowance\n4.PF Amount.
Just show the result, please don't show any calcualtions""")
print(result)

Sure! Here are the percentages of the mentioned components with respect to Basic:

1. HRA: 39.99%
2. Conveyance: 6.47%
3. Special Allowance: 129.20%
4. PF Amount: 12.02%


In [11]:
result = conversation.predict(input="What could be the total tax to be paid for the entire fiscal year?")
print(result)

To calculate the total tax to be paid for the entire fiscal year, we need to consider the Year-to-Date (YTD) values for the Income Tax on Salary. 

From the pay slip data you provided, the YTD value for Income Tax on Salary is Rs. 1,167.00.

Please note that without additional information about the tax brackets, deductions, and exemptions, it is not possible to accurately determine the total tax to be paid for the entire fiscal year. It would be best to consult with a tax professional or refer to the relevant tax regulations for an accurate calculation.


In [15]:
result = conversation.predict(input="What could be the total tax to be paid for the entire fiscal year? Hint: Multiply Income Tax on Salary per Month * 12")
print(result)

To calculate the total tax to be paid for the entire fiscal year, we can use the hint provided:

Total Tax = Income Tax on Salary per Month * 12

From the pay slip data you provided, the Income Tax on Salary is Rs. 1,167.00. Assuming this value is the monthly tax, we can calculate the total tax for the entire fiscal year:

Total Tax = 1,167.00 * 12 = Rs. 14,004.00

Please note that this calculation is based on the assumption that the provided value of Rs. 1,167.00 is the monthly tax. However, without additional information about the tax brackets, deductions, and exemptions, this calculation may not be accurate. It is always recommended to consult with a tax professional or refer to the relevant tax regulations for an accurate calculation of the total tax for the entire fiscal year.


In [18]:
result = conversation.predict(input="Verify whether the Net Transfer is correct or wrong?")
print(result)

To verify whether the Net Transfer amount is correct or wrong, we need to calculate it based on the provided pay slip data.

From the pay slip data you provided, the Gross Earnings is Rs. 68,256.00 and the Gross Deductions is Rs. 4,881.00.

To calculate the Net Transfer, we subtract the Gross Deductions from the Gross Earnings:

Net Transfer = Gross Earnings - Gross Deductions
               = Rs. 68,256.00 - Rs. 4,881.00
               = Rs. 63,375.00

According to the provided pay slip data, the Net Transfer amount is Rs. 63,375.00.

If you have any specific concerns or doubts about the Net Transfer amount, please let me know and I'll be happy to assist you further.


In [19]:
result = conversation.predict(input="I need Net Transfer in words")
print(result)

Based on the provided pay slip data, the Net Transfer amount is Rs. 63,375.00. In words, it can be written as "Rupees Sixty Three Thousand Three Hundred And Seventy Five Only."
